In [ ]:
%pip install transformers==4.31.0
%pip install peft==0.4.0
%pip install accelerate==0.21.0
%pip install bitsandbytes==0.40.2
%pip install safetensors==0.3.3 # 제거 가능할 수 있습니다.
%pip install tokenizers==0.13.3 # 제거 가능할 수 있습니다.
%pip install datasets==2.14.1

In [ ]:
import os
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import torch

import bitsandbytes as bnb

## 아마존 세이지메이커 스튜디오에서 QLoRA로 LLaMA 7B 미세 조정

우리는 Tim Dettmers et al.의 논문 "[QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation](https://arxiv.org/abs/2106.09685)"에서 최근에 소개된 방법을 활용할 것입니다. QLoRA는 미세 조정하는 동안 대규모 언어 모델의 메모리 사용량을 줄이는 새로운 기술로 성능이 줄어들지 않습니다. QLoRA의 간단한 설명은 다음과 같습니다.

* 사전 학습된 모델을 4비트로 양자화하고 고정합니다.
* 작은, 학습 가능한 어댑터 레이어를 추가합니다. (LoRA)
* 어댑터 레이어만 미세 조정하고, 고정된 양자화된 모델을 컨텍스트로 사용합니다.

### 하드웨어 요구 사항

우리는 다양한 모델 크기에 적합한 인스턴스 유형을 결정하기 위해 여러 실험을 수행했습니다. 다음 표는 우리의 실험 결과를 보여줍니다. 표에는 인스턴스 유형, 모델 크기, 컨텍스트 길이 및 최대 배치 크기가 포함되어 있습니다.

| 모델        | 인스턴스 유형     | 최대 배치 크기 | 콘텍스트 길이 |
|--------------|-------------------|----------------|----------------|
| [LLama 7B]() | `(ml.)g5.4xlarge` | `3`            | `2048`         |
| [LLama 13B]() | `(ml.)g5.4xlarge` | `2`            | `2048`         |
| [LLama 70B]() | `(ml.)p4d.24xlarge` | `1++` (need to test more configs)            | `2048`         |


> `g5.4xlarge` 인스턴스 유형 대신 `g5.2xlarge`를 사용할 수 있지만, `merge_weights` 매개변수를 사용할 수 없습니다. LoRA 가중치를 모델 가중치에 병합하려면 모델이 메모리 크기에 적절해야 합니다. 하지만 학습 후 어댑터 가중치를 저장하고 [merge_adapter_weights.py](./scripts/merge_adapter_weights.py)를 활용하여 병합할 수 있습니다.

_참고: 이 목록은 미래에 확장할 계획입니다. 여러분의 설정도 기여 부탁드립니다!_

In [ ]:
import argparse
parser = argparse.ArgumentParser()

# 모델 ID 및 데이터 세트 경로 인수 추가
parser.add_argument(
    "--model_id",
    type=str,
    default="NousResearch/Llama-2-7b-hf", # 게이트 아님, # TODO: 13b 시도하기
    help="Model id to use for training.",
)
parser.add_argument(
    "--dataset_path", 
    type=str, 
    default="lm_dataset", 
    help="Path to dataset."
)
# parser.add_argument(
#     "--hf_token", 
#     type=str, 
#     default=HfFolder.get_token(), 
#     help="Path to dataset."
# )
# 에포크, 배치 크기, 학습률, 시드를 위한 학습 하이퍼파라미터 추가
parser.add_argument(
    "--epochs", 
    type=int, 
    default=1, 
    help="Number of epochs to train for."
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=1,
    help="Batch size to use for training.",
)
parser.add_argument(
    "--lr", 
    type=float, 
    default=5e-5, 
    help="Learning rate to use for training."
)
parser.add_argument(
    "--seed", 
    type=int, 
    default=42, 
    help="Seed to use for training."
)
parser.add_argument(
    "--gradient_checkpointing",
    type=bool,
    default=True,
    help="Path to deepspeed config file.",
)
parser.add_argument(
    "--bf16",
    type=bool,
    default=True if torch.cuda.get_device_capability()[0] >= 8 else False,
    help="Whether to use bf16.",
)
parser.add_argument(
    "--merge_weights",
    type=bool,
    default=True,
    help="Whether to merge LoRA weights with base model.",
)
args, _ = parser.parse_known_args()

# if args.hf_token:
#     print(f"Logging into the Hugging Face Hub with token {args.hf_token[:10]}...")
#     login(token=args.hf_token)

In [ ]:
# https://github.com/artidoro/qlora/blob/main/qlora.py에서 복사
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


# https://github.com/artidoro/qlora/blob/main/qlora.py에서 복사
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


def create_peft_model(model, gradient_checkpointing=True, bf16=True):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )
    from peft.tuners.lora import LoraLayer

    # INT4 모델 학습을 준비합니다.
    model = prepare_model_for_kbit_training(
        model, use_gradient_checkpointing=gradient_checkpointing
    )
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # lora 대상 모듈 가져오기
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    model = get_peft_model(model, peft_config)

    # 모델을 float32로 레이어 norms을 업캐스팅하여 전처리합니다.
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)

    model.print_trainable_parameters()
    return model

## 데이터 세트 로드 및 준비

우리는 [dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k) 데이터 세트를 활용할 것입니다. 이 데이터 세트는 Databricks 직원들이 생성한 다양한 행동 카테고리의 지침에 따른 레코드입니다. 이 카테고리는 [InstructGPT 논문](https://arxiv.org/abs/2203.02155)에서 설명된 바와 같이 브레인스토밍, 분류, 닫힌 QA, 생성, 정보 추출, 개방형 QA, 요약 등을 포함합니다.

```python
{
  "instruction": "What is world of warcraft",
  "context": "",
  "response": "World of warcraft is a massive online multi player role playing game. It was released in 2004 by bizarre entertainment"
}
```

`samsum` 데이터 세트을 로드하려면 🤗 데이터세트 라이브러리의 load_dataset() 메서드를 사용합니다.

In [ ]:
# seed 설정
set_seed(args.seed)

from datasets import load_dataset
from random import randrange

# 허브에서 데이터 세트 로드
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = dataset.select(range(1000))

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

모델을 지침에 맞게 조정하려면 구조화된 예제를 지침으로 설명된 작업 집합으로 변환해야 합니다. `formatting_function`을 정의하여 샘플을 받아 우리가 정의한 형식의 문자열을 반환합니다.

In [ ]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # 모든 부분을 하나로 결합합니다.
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt


In [ ]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

In [ ]:
from transformers import AutoTokenizer

#model_id = "meta-llama/Llama-2-13b-hf" # 조각화된 가중치, 게이트
model_id = "NousResearch/Llama-2-7b-hf" # 게이트 아님, TODO: 13b 시도하기
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from random import randint
from itertools import chain
from functools import partial


# 각 샘플에 프롬프트를 추가하기 위한 템플릿 데이터 세트
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# 각 샘플에 프롬프트 템플릿 적용
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# 무작위 샘플 출력
print(dataset[randint(0, len(dataset))]["text"])

# 다음 배치에서 사용할 나머지를 저장할 빈 리스트
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # 정의된 전역 remainder 변수에 다음 배치에서 사용하기 위한 나머지 저장
    global remainder
    # 모든 텍스트를 연결하고 이전 배치의 나머지를 추가
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # 배치의 총 토큰 수 가져오기
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # 배치의 최대 청크 수 얻기
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # 최대 길이의 청크로 나누기
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # 다음 배치에서 사용할 나머지를 전역 변수에 추가
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # 레이블 준비
    result["labels"] = result["input_ids"].copy()
    return result


# 데이터 세트를 토큰화하고 청크로 나누기
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# 총 샘플 수 출력
print(f"Total number of samples: {len(lm_dataset)}")

In [ ]:
# 위의 청크 처리는 행의 수를 줄입니다.
print(lm_dataset)

In [ ]:
# BNB 구성으로 허브에서 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    args.model_id,
    use_cache=False
    if args.gradient_checkpointing
    else True,  # 그래디언트 체크포인팅을 위해 필요합니다.
    device_map="auto",
    quantization_config=bnb_config,
)

# PEFT 구성 파일 생성
model = create_peft_model(
    model, gradient_checkpointing=args.gradient_checkpointing, bf16=args.bf16
)

In [ ]:
# 학습 인수 정의
output_dir = "./tmp/llama2_qlora"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=args.per_device_train_batch_size,
    bf16=args.bf16,  # BF16이 가능할 경우 사용
    learning_rate=args.lr,
    num_train_epochs=args.epochs,
    gradient_checkpointing=args.gradient_checkpointing,
    # 로깅 전략 설정
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
)

# Trainer 인스턴스 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=default_data_collator,
)

# 학습 시작
trainer.train()

In [ ]:
adapter_save_dir = "./llama2_qlora_adapter"

tokenizer.save_pretrained(adapter_save_dir)

# 어댑터 가중치를 기본 모델과 병합한 후 저장
# INT4 모델 저장
trainer.model.save_pretrained(
    adapter_save_dir, safe_serialization=False
)

# 메모리 정리
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
from peft import AutoPeftModelForCausalLM

merged_save_dir = "./llama2_qlora_merged"

# 추론을 쉽게 하기 위해 토크나이저 저장
tokenizer.save_pretrained(merged_save_dir)

# FP16으로 PEFT 모델 로드
model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_save_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
)  

# LoRA와 기본 모델 병합 후 저장
model = model.merge_and_unload()        
model.save_pretrained(
    merged_save_dir, safe_serialization=True, max_shard_size="2GB"
)